# Euclidean Geometry by high-peformance SMT solvers?

### Siddhartha Gadgil and Anand Rao Tadipatri

Famous results of G&ouml;del, Turing, Church and others show that there is no algorithm that can take a mathematical statement as input and return a proof or disproof. Yet, by a result of Tarski, we do have such programs for statements in Euclidean geometry (more precisely _first-order_ statements). 

This notebook is to demonstrate the results of our attempts using [`Z3`](https://github.com/Z3Prover/z3) &mdash; an open source high-performance solver from Microsoft. This notebook is a companion to our [article](http://math.iisc.ac.in/~gadgil/SMTGeom.pdf), which has more background.
 

In [1]:
from z3 import *
set_param(proof=True)

---
## Warmup: A simple problem

As a warmup and sanity check, consider
the problem of showing that for an arbitrary point $P = (x, y)$, the three points
$P = (x, y)$, $O = (0, 0)$ and $−P = (−x, −y)$ are collinear.

In [2]:
P = (x, y) = Reals('x y')  #the coordinates of the point P
O = (0, 0)  #the coordinates of the origin
Q = (-x, -y)  #the reflection of the point P about the origin

#### Equations for collinearity

The condition for three points $(x_1, y_1), (x_2, y_2) \text{ and } (x_3, y_3)$ being collinear is 

$$
\frac{y_2 - y_1}{x_2 - x_1} = \frac{y_3 - y_1}{x_3 - x_1}
$$

Intuitively, this says that the slope of the line joining the points $(x_1, y_1)$ and $(x_2, y_2)$ is equal to the slope of the line joining $(x_1, y_1)$ and $(x_3, y_3)$.

The above expression is equivalent to

$$(y_2 - y_1) \cdot (x_3 - x_1) = (y_3 - y_1) \cdot (x_2 - x_1)$$

In [3]:
def are_collinear(p, q, r):
    """
    Checks if three points - `p`, `q`, `r` - are collinear.
    
    Here, `p[0]` and `p[1]` denote the _x_ and _y_ coordinates of `p` respectively.
    """
    return ( (q[1]-p[1])*(r[0]-p[0]) == (r[1]-p[1])*(q[0]-p[0]) )

In [4]:
prove((are_collinear(P, O, Q)))

proved


This shows that the claim that the points $P$, $O$ and $Q$ are collinear is true.

Internally, the `prove` function works roughly in the following way:
- The given claim (that the points `P`, `O` and `Q` are collinear is first negated.
- The solver then checks whether the given system of equations is satisfiable, i.e., whether there are real numbers `x` and `y` for which `Not(are_collinear((x, y), (0, 0), (-x, -y))` holds.
- If no such solutions are found, this shows by contradiction that the initial claim was correct, namely that for any point `(x, y)`, the points `(x, y), (0, 0), (-x, -y)` are collinear.

In [5]:
collinearity_solver = Solver()
collinearity_solver.add(Not(are_collinear(P, O, Q)))  #the negation of the statement
collinearity_solver.check()  #`unsat` indicates that the given equation is not satisfiable

unsat

This is the statement given to the solver

In [6]:
collinearity_solver  #the statement of the claim

[Not((0 - y)*(-x - x) == (-y - y)*(0 - x))]

This is the code in SMT2 format

In [7]:
print(collinearity_solver.sexpr())  #this is how the code is represented in SMT2 format

(declare-fun x () Real)
(declare-fun y () Real)
(assert (let ((a!1 (= (* (- 0.0 y) (- (- x) x)) (* (- (- y) y) (- 0.0 x)))))
  (not a!1)))



One can also use the `Z3` solver to produce a proof of the result using the solver, which can then be verified independently to ensure that it is correct.

In [8]:
collinearity_solver.proof()

mp(asserted(Not((0 - y)*(-x - x) == (-y - y)*(0 - x))),
   trans(monotonicity(trans(monotonicity(trans(trans(monotonicity(trans(rewrite(0 -
                                        y ==
                                        0 + -1*y),
                                        rewrite(0 + -1*y ==
                                        -1*y),
                                        0 - y == -1*y),
                                        trans(trans(monotonicity(rewrite(-x ==
                                        -1*x),
                                        -x - x == -1*x - x),
                                        rewrite(-1*x - x ==
                                        -1*x + -1*x),
                                        -x - x ==
                                        -1*x + -1*x),
                                        rewrite(-1*x + -1*x ==
                                        -2*x),
                                        -x - x == -2*x),
                                        (0 - y)*(-x - x) ==
                                        -1*y*-2*x),
                                        rewrite(-1*y*-2*x ==
                                        2*y*x),
                                        (0 - y)*(-x - x) ==
                                        2*y*x),
                                        rewrite(2*y*x ==
                                        2*x*y),
                                        (0 - y)*(-x - x) ==
                                        2*x*y),
                                        trans(trans(monotonicity(trans(trans(monotonicity(rewrite(-y ==
                                        -1*y),
                                        -y - y == -1*y - y),
                                        rewrite(-1*y - y ==
                                        -1*y + -1*y),
                                        -y - y ==
                                        -1*y + -1*y),
                                        rewrite(-1*y + -1*y ==
                                        -2*y),
                                        -y - y == -2*y),
                                        trans(rewrite(0 - x ==
                                        0 + -1*x),
                                        rewrite(0 + -1*x ==
                                        -1*x),
                                        0 - x == -1*x),
                                        (-y - y)*(0 - x) ==
                                        -2*y*-1*x),
                                        rewrite(-2*y*-1*x ==
                                        2*y*x),
                                        (-y - y)*(0 - x) ==
                                        2*y*x),
                                        rewrite(2*y*x ==
                                        2*x*y),
                                        (-y - y)*(0 - x) ==
                                        2*x*y),
                                        ((0 - y)*(-x - x) ==
                                        (-y - y)*(0 - x)) ==
                                        (2*x*y == 2*x*y)),
                            rewrite((2*x*y == 2*x*y) == True),
                            ((0 - y)*(-x - x) ==
                             (-y - y)*(0 - x)) ==
                            True),
                      Not((0 - y)*(-x - x) ==
                          (-y - y)*(0 - x)) ==
                      Not(True)),
         rewrite(Not(True) == False),
         Not((0 - y)*(-x - x) == (-y - y)*(0 - x)) == False),
   False)

---
## Pappus' theorem

In addition to being a typical geometry result, *Pappus' hexagon theorem*
has a deeper mathematical meaning (corresponding to commutativity for
affine geometries over division rings), as you can read in the beautiful book *Geometric Algebra* by Emil Artin.

Suppose we are given two lines, with points $a$, $b$ and $c$ on
the first line and $A$, $B$ and $C$ on the second line as in
figure below. We consider the general case, where no pair of lines
involving these points are parallel. Let $P$ be the intersection of
the lines $Ab$ and $aB$, $Q$ the intersection of $Ac$ and
$aC$, and $R$ the intersection of $Bc$ and $bC$.

The Pappus
hexagon theorem is the result that $P$, $Q$ and $R$ are
*collinear*, i.e., there is a line containing all three of these
points, for all choices of $a$, $b$, $c$, $A$, $B$, and $C$
of the above form.


![Pappus Theorem](../tex/Pappus.png)

### Formulating Pappus' theorem in terms of polynomial (in)equalities

We shall formulate the Pappus theorem in terms of collinearity, and
then translate this into equations and inequations.
Recall that collinearity can be
expressed as a polynomial equality. Namely, points with coordinates
$(x_1, y_1)$, $(x_2, y_2)$ and $(x_3, y_3)$, which we assume to be
distinct, are collinear if and only if
$\frac{y_2 - y_1}{x_2 - x_1} = \frac{y_3 - y_1}{x_3 - x_1}$ which is
equivalent to $(y_2 - y_1)(x_3 - x_1) = (y_3 - y_1)(x_2 - x_1)$.

We next translate the Pappus theorem, first into coordinate geometry and then into
real equations and inequations.

#### Choosing coordinates

While one can (and we initially did) take arbitrary coordinates for the
$6$ points $a$, $b$, $c$, $A$, $B$ and $C$ and add
equations for their being collinear, we consider a simpler variant where
we choose coordinates and parametrize the points. 

Namely, we can take
$a$, $b$ and $c$ on the $x$-axis with $a = (1, 0)$. Then we have
$b = (1 + u, 0)$ and $c = (1 + u + v, 0)$ with $u>0$ and $v>0$.


In [9]:
u, v = Reals('u v')
a, b, c = (1, 0), (1+u, 0), (1+u+v, 0)  #the points on the first line

Similarly, if we let $A = (x_A, y_A)$, then we can assume that
$B = (x_A(1+ U), y_A(1 + U))$ for some $U > 0$ and
$C = (x_A(1+ U + V), y_A(1 + U + V))$ for some $V > 0$. Further, we
can assume that $y_A > 0$.

In [10]:
U, V = Reals('U V')  #the scaling parameters

#the points on the second line
A = (x_A, y_A) = Reals('x_A y_A')
B = (x_B, y_B) = (x_A*(1+U), y_A*(1+U))
C = (x_C, y_C) = (x_A*(1+U+V), y_A*(1+U+V))

Let the points $P= (x_P, y_P)$, $Q = (x_Q, y_Q)$ and
$R= (x_R, y_R)$ have arbitrary coordinates. We add equations
corresponding to their being intersection points, as we see below. 

In [11]:
P = (x_P, y_P) = Reals('x_P y_P')
Q = (x_Q, y_Q) = Reals('x_Q y_Q')
R = (x_R, y_R) = Reals('x_R y_R')

Thus,
we have $12$ variables in all, $6$ of them the parameters $u$,
$v$, $x_A$, $y_A$, $U$ and $V$ for the problem and $6$ more
coordinates of the intersection points. Further, we have inequations
$u >0$, $v >0$, $y_A >0$, $U > 0$ and $V >0$. We shall add to
these equations and inequations from the statement of the theorem.

#### Equations and inequations

We reformulate the Pappus hexagon theorem in terms of collinearity.
Observe that $P$ being the intersection point of $Ab$ and $aC$ is
equivalent to both the triples of points $(A, P, b)$ and $(a, P, B)$
being collinear. We have similar conditions for $Q$ and $R$. Thus,
the conditions on $P$, $Q$ and $R$ can be formulated in terms of
collinearity of $6$ triples of points.

In [12]:
def is_intersection_point(p, X, y, x, Y):
    """
    Gives the condition for the point `p` to lie on the intersection of the lines `XY` and `xy`.
    """
    return And(are_collinear(p, X, y), are_collinear(p, x, Y))

Finally, Pappus' theorem can be stated as

In [13]:
pappus_theorem = Implies(And(
    is_intersection_point(P, A, b, a, B),
    is_intersection_point(Q, B, c, b, C),
    is_intersection_point(R, C, a, c, A)),
                         are_collinear(P, Q, R))

We seek to prove this theorem by contradiction, namely we add the condition that
the the theorem is false, and show that the resulting system cannot be
satisfied.

In [14]:
set_param(proof = False)

pappus_solver = Solver()

pappus_solver.add(u > 0); pappus_solver.add(v > 0)
pappus_solver.add(U > 0); pappus_solver.add(V > 0)

pappus_solver.add(Not(pappus_theorem))

pappus_solver.check()

unsat

As before, since the negation of the theorem is unsatisfiable, the theorem must be true.

The first line of the above cell - `set_param(proof = False)` - asks the solver to check satisfiability without trying to produce a proof. If one asks for a proof by changing the line to - `set_param(proof = True)` - the solver times out after a few seconds and returns `unknown`.

Thus, the `Z3` solver can be used to *solve*, but not *prove* Pappus' theorem.

---
## Menelaus's theorem

Another classical result in plane geometry is *Menelaus's theorem*, formulated by Menelaus of Alexandria.

Consider a triangle with vertices $A$, $B$ and $C$ and a line that crosses the (possibly extended) edges $AB$, $BC$ and $CA$ at points $D$, $E$ and $F$ respectively. Menelaus' theorem states that

$$
\frac{DA}{DB} \times \frac{EB}{EC} \times \frac{FC}{FA} = 1
$$

![Menelaus' theorem](../tex/MenelausIllustration.png)

This theorem also has a converse, provided certain additional conditions are satisfied - 
if $D$, $E$ and $F$ are points on the (possibly extended) edges of the triangle $ABC$, and either exactly one or three of these points are contained on extended edges, and the points satisfy the above equation, then they are collinear.

Note that the additional condition of either exactly one or three of the points being contained on extended edges is automatically satisfied in the first statement.

The above theorem is also valid when signed distances are used in place of regular distances (with signed distances, the line segments $AB$ and $BA$ have the same length but differ in sign). In this case, the extra condition is not required for the converse to be true.

### Formulating Menelaus's theorem in terms of polynomial (in)equations

Like Pappus' theorem, Menelaus' theorem is solved by formulating it in coordinate geometry and a system of equations in real variables.

#### Defining the variables

Three distinct points representing the vertices of the triangle $ABC$ are arbitrarily chosen (one could optimise this by scaling and shifting the triangle to make two of the three vertices coincide with points $(0, 0)$ and $(1, 1)$ on the plane).

In [15]:
#A, B, C are the vertices of the triangle
A = (x_a, y_a) = Reals('x_a y_a')
B = (x_b, y_b) = Reals('x_b y_b')
C = (x_c, y_c) = Reals('x_c y_c')

A line passing through two points $P$ and $Q$ on the plane can be parameterised by a single real number $l$. Moreover, this parameterisation can be chosen such that the value $l = 0$ corresponds to the point $P$ and $l = 1$ corresponds to the point $Q$.

The three edges of the triangle - $AB$, $BC$, $CA$ - can be parameterised in this manner by real numbers $r$, $s$ and $t$.

In [16]:
"""
Each of the edges of the triangle can be parameterised by a single variable,
which is equal to the first vertex at 0 and equal to the second vertex at 1.
"""

r, s, t = Reals('r s t')  #the parameters

Since the theorem requires the transversal line to cross all three edges of the triangle, the points of intersection - $D$, $E$, $F$ - correspond to some real values of the parameters $r$, $s$ and $t$.

In [17]:
def cut(l, P, Q):
    """
    With the line PQ parameterised as described above,
    this function returns the point R obtained when the parameter is equal to `l`.
    """
    return (P[0] + l*(Q[0] - P[0]), P[1] + l*(Q[1] - P[1]))

# D, E, F are points on the edges AB, BC, CA respectively
D, E, F = cut(r, A, B), cut(s, B, C), cut(t, C, A)

After defining the nine variables (six for the vertices of the triangle and three for the parameters of the edges) required to describe the setup in coordinate geometry, the next step is to formulate a system of equations describing the theorem.

#### Equations and Inequations

For the "forward" part of the theorem statement, the hypothesis is that the three points of intersection - $D$, $E$ and $F)$ - are collinear. As mentioned above, collinearity can be formulated as a polynomial equation in terms of the coordinates of the three points.

In [18]:
are_collinear(D, E, F)

(y_b + s*(y_c - y_b) - (y_a + r*(y_b - y_a)))*
(x_c + t*(x_a - x_c) - (x_a + r*(x_b - x_a))) ==
(y_c + t*(y_a - y_c) - (y_a + r*(y_b - y_a)))*
(x_b + s*(x_c - x_b) - (x_a + r*(x_b - x_a)))

The statement of Menelaus' theorem holds even when the regular Euclidean distances (given by $\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$ for points $(x_1, y_1)$ and $(x_2, y_2)$) are replaced by the squares of the Euclidean distances ($(x_1 - x_2)^2 + (y_1 - y_2)^2$). The statement can also be rewritten as

$$
DA \times EB \times FC = DB \times EC \times FA 
$$

With these simplifications, the forward part of the theorem can be expressed as a polynomial in all the variables involved.

In [19]:
def d(p, q):
    """
    Returns the square of the Euclidean distance between two points.
    """
    return (p[0] - q[0])**2 + (p[1] - q[1])**2

dist_eq = d(D, A) * d(E, B) * d(F, C) == d(D, B) * d(E, C) * d(F, A)

The converse (or "reverse") statement has the additional requirement that the exactly either one or three of the intersection points lie on the extensions of edges of the triangle. Due to the way the parameterisation of the lines was chosen, a point lies on an extended edge if and only if the corresponding value of the parameter is not contained in the $[0, 1]$ interval.

The condition of a parameter $l$ being contained in the $[0, 1]$ interval can be formulated in terms of inequations

$$
(0 \lt l) \wedge (l \lt 1)
$$

The converse of the theorem requires that out of the three parameters $r$, $s$, and $t$, an odd number of them (either one or three) must *not* satisfy the above condition.

This can be captured using the `XOR` (exclusive `OR`, which is `True` when exactly one of the inputs is `True`) function ($\oplus$) -

$$
\neg \left(\left( (0 \lt r) \wedge (r \lt 1) \right) \oplus \left( (0 \lt s) \wedge (s \lt 1) \right) \oplus \left( (0 \lt t) \wedge (t \lt 1) \right)\right)
$$

which is `True` only when an odd number of intersection points lie on the extended edges.

In [20]:
def in_bounds(l):
    """
    Checks whether the parameter is within the range (0, 1),
    i.e, whether the point corresponding to the parameter value `l` is 
    contained within the corresponding edge or on an extension of it.
    """
    return And(0 < l, l < 1)

odd_in_bounds = Xor(Xor(in_bounds(r), in_bounds(s)), in_bounds(t))

The forward and reverse implication parts of the theorem statement can now be formulated as follows

In [21]:
menelaus_theorem_fwd = Implies(And(Not(odd_in_bounds), dist_eq), are_collinear(D, E, F))
menelaus_theorem_rev = Implies(And(Not(odd_in_bounds), are_collinear(D, E, F)), dist_eq)

menelaus_theorem = And(menelaus_theorem_fwd, menelaus_theorem_rev)

In [22]:
set_param(proof = False)

menelaus_solver = Solver()

menelaus_solver.add(Not(are_collinear(A, B, C)))
menelaus_solver.add(Not(menelaus_theorem))

menelaus_solver.check()

unsat

As with Pappus' theorem, the `Z3` solver can solve (but not prove) Menelaus' theorem.

#### Using signed distances (optional)

As mentioned above, the theorem can also be formulated using signed distances. This can be achieved easily using the values of the three parameters $r$, $s$ and $t$.

Notice that the origin formula uses only depends on the ratios of the distances of the intersection point from the vertices, and not the actual distances themselves. The parameterisation of the lines was chosen such that the value $0$ corresponds to the first vertex and the value of $1$ corresponds to the second vertex.

For a line $PQ$ parameterised by a variable $l$, the (unsigned) distance between two points on the line corresponding to the values $l_1$ and $l_2$ of the parameter is given by 

$$
    \vert l_2 - l_1 \vert \cdot d(P, Q)
$$,
and the signed distance can be calculated by replacing $\vert l_2 - l_1 \vert$ in the above formula by just $\left(l_2 - l_1\right)$.

For a point $R$ on the line $PQ$ with the parameter value equal to $l_R$, the signed distance from $P$ is given by $\left(l_R - 0\right) \cdot d(P, Q)$, and likewise, the signed distance from $Q$ is given by $\left(l_R - 1\right) \cdot d(P, Q)$.

The signed version of the distance equation can now be formulated as

$$
(r - 0) \cdot (s - 0) \cdot (t - 0) = (r - 1) \cdot (s - 1)\cdot (t - 1)
$$

In [23]:
signed_dist_eq = (r - 0) * (s - 0) * (t - 0) == (r - 1) * (s - 1) * (t - 1)

and the final form of Menelaus' theorem formulated in terms of distances is

In [24]:
signed_menelaus_theorem = And(Implies(are_collinear(D, E, F), signed_dist_eq), 
                              Implies(signed_dist_eq, are_collinear(D, E, F)))

In [25]:
set_param(proof = False)

signed_menelaus_solver = Solver()
signed_menelaus_solver.add(Not(are_collinear(A, B, C)))
signed_menelaus_solver.add(Not(signed_menelaus_theorem))

signed_menelaus_solver.check()

unsat

As expected, the solver returns `unsat`.

---

The two theorems mentioned in this article can also be explored interactively as GeoGebra applets. Here are links to two applets developed by GeoGebra users `Don_Abucito` and `chisanga` respectively:

- [Exploration of Pappus Hexagon theorem](https://www.geogebra.org/m/fhf56hbu)
- [Exploration of Menelaus's theorem](https://www.geogebra.org/m/TAfxCPT9)

---